# Przewidywanie zużycia energii z tensorflow 2.4

### Niezbędne biblioteki

In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
from datetime import datetime

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import preprocessing

### Importowanie danych

In [4]:
data_date = pd.read_csv("daily_weather_and_energy_consumption.csv")

In [5]:
data_date.head()

,Data,Temperatura,Wilogtność powietrza,Prędkość wiatru,Zachmurzenie,Dobowa prognoza zapotrzebowania KSE,Rzeczywiste zapotrzebowanie KSE
0,2015-01-01,-0.77,31.80,1.34,7.57,NaN,15098.40
1,2015-01-02,1.27,29.94,2.47,6.83,17822.96,17869.20
2,2015-01-03,1.87,28.00,2.80,6.95,17639.33,17684.92
3,2015-01-04,0.42,28.89,2.79,6.27,16235.42,16453.76
4,2015-01-05,-1.03,29.75,1.91,6.05,18892.71,18856.44


Jako, że zapotrzebowanie intuicyjnie może zależeć od miesiąca (grudzień vs czerwiec), ale nie od samego roku (np. 2016 bo sama ta informacja nie wystarcza, żeby coś wnioskować), zamieniam kolumnę <b>Data</b> na procent roku (co koreluje z porą roku)

In [6]:
def date_to_double(dateS, form):
    d = datetime.strptime(dateS, form)
    
    beg = datetime(d.year,1,1,0,0,0)
    end = datetime(d.year,12,31,23,59,59)
    
    sec = (d-beg).total_seconds()
    
    return sec / (end-beg).total_seconds()

In [7]:
data = data_date.copy()
data["Data"] = data["Data"].map(lambda d: date_to_double(d, "%Y-%m-%d"))
data = data.drop(['Dobowa prognoza zapotrzebowania KSE'], axis=1)

data.dropna()
data.head()

,Data,Temperatura,Wilogtność powietrza,Prędkość wiatru,Zachmurzenie,Rzeczywiste zapotrzebowanie KSE
0,0.000000,-0.77,31.80,1.34,7.57,15098.40
1,0.002740,1.27,29.94,2.47,6.83,17869.20
2,0.005479,1.87,28.00,2.80,6.95,17684.92
3,0.008219,0.42,28.89,2.79,6.27,16453.76
4,0.010959,-1.03,29.75,1.91,6.05,18856.44


Żeby uczenie przebiegało lepiej należy znormalizować dane

In [47]:
def normalise( tensor ):
    
    x = tensor.values[:]
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(data=x_scaled, columns=tensor.columns)

data_norm = normalise(data)

Dane należy podzielić na dane wejściowe i wyjściowe

In [48]:
features = data_norm.loc[0:,'Data':'Zachmurzenie']
labels   = data_norm.loc[0:, 'Rzeczywiste zapotrzebowanie KSE']

oraz na zbiór uczący i zbiór testowy

In [49]:
features_train = features.sample(frac=0.8, random_state=200)
features_test  = features.drop(features_train.index)

labels_train   = labels.sample(frac=0.8, random_state=200)
labels_test    = labels.drop(labels_train.index)

In [50]:
features.head()

,Data,Temperatura,Wilogtność powietrza,Prędkość wiatru,Zachmurzenie
0,0.000000,0.383945,0.640147,0.198157,0.944751
1,0.002747,0.431968,0.564259,0.458525,0.842541
2,0.005494,0.446092,0.485108,0.534562,0.859116
3,0.008242,0.411959,0.521420,0.532258,0.765193
4,0.010989,0.377825,0.556508,0.329493,0.734807


In [51]:
labels.head()

0    0.227735
1    0.477655
2    0.461033
3    0.349986
4    0.566702
Name: Rzeczywiste zapotrzebowanie KSE, dtype: float64

### Definiowanie warstw

In [61]:
leaky_relu_alpha = 0.2
dropout_rate = 0.5
learning_rate = 0.07

def dense( inputs , weights ):
    '''
        tf.nn.leaky_relu: Leaky version of a Rectified Linear Unit.
                          A leaky ReLU layer performs a threshold operation, 
                          where any input value less than zero is multiplied by a fixed scalar.
        tf.nn.dropout:    Computes dropout: randomly sets elements to zero to prevent overfitting.
        tf.matmul:        Multiplies matrix a by matrix b, producing a * b.
    '''
    x = tf.nn.leaky_relu( tf.matmul( inputs , weights ) , alpha=leaky_relu_alpha )
    return x #tf.nn.dropout( x , rate=dropout_rate )

### Inicializowanie sieci

In [54]:
# Initialiser creates tensor of numbers with uniform distribution
initializer = tf.initializers.glorot_uniform()

def get_weight( shape , name ):
    # tf.Variable changes a tensor into a constant variable
    return tf.Variable( initializer( shape ) , name=name , trainable=True , dtype=tf.float32 )

<i><b>shapes</b></i> tutaj oznacza, że warstwy gęste mają kolejno 5, 20, 10, 10 i 1 neuronów

In [67]:
shapes = [
    [ 5 , 20 ] , 
    [ 20 , 10 ] ,
    [ 10 , 10 ] ,
    [ 10 , 1 ] ,
]

weights = []
for i in range( len( shapes ) ):
    weights.append( get_weight( shapes[ i ] , 'weight{}'.format( i ) ) )

### Tworzenie modelu
Składa się on z warstwy wejściowej, kilku warstw gęstych i warstwy wyjściowej

In [60]:
def model( x ) :
    x = tf.cast( x , dtype=tf.float32 ) # Element-wise cast to float32
    
    d1 = dense(  x , weights[ 0 ] )
    d2 = dense( d1 , weights[ 1 ] )
    d3 = dense( d2 , weights[ 2 ] )
    
    logits = tf.matmul( d3 , weights[ 3 ] )
    
    return tf.nn.softmax( logits ) # softmax activations

### Uczenie

funkcja błędu

In [88]:
def loss( pred , target ):
    return tf.losses.mean_squared_error( target , pred )

In [90]:
optimizer = tf.optimizers.Adam( learning_rate )

def train_step( model, inputs , outputs ):
    with tf.GradientTape() as tape:
        current_loss = loss( model( inputs ), outputs)
    grads = tape.gradient( current_loss , weights )
    optimizer.apply_gradients( zip( grads , weights ) )
#     print( tf.reduce_mean( current_loss ) )
    
num_epochs = 10

for e in range( num_epochs ):
    print(f'epoch {e}')
    for feature, label in zip(features_train.iloc, labels_train):
        train_step( model , np.matrix(feature.values) , np.matrix(label) )

epoch 0
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9


In [91]:
for feature, label in zip(features_test.iloc, labels_test):
    print(f'prediction = {model( np.matrix(feature.values) ).numpy()[0][0]}, true={label}')

prediction = 1.0, true=0.39164517708945024
prediction = 1.0, true=0.7072420551702618
prediction = 1.0, true=0.5762751459622599
prediction = 1.0, true=0.517527647818075
prediction = 1.0, true=0.7069885999258578
prediction = 1.0, true=0.6027057474810069
prediction = 1.0, true=0.6580582081165212
prediction = 1.0, true=0.32381117313699126
prediction = 1.0, true=0.6576947118051515
prediction = 1.0, true=0.6586535122492847
prediction = 1.0, true=0.6884656216373211
prediction = 1.0, true=0.09320658767161238
prediction = 1.0, true=0.5289854484218581
prediction = 1.0, true=0.5824482694238271
prediction = 1.0, true=0.578771815416032
prediction = 1.0, true=0.5600440885848632
prediction = 1.0, true=0.35886827272505917
prediction = 1.0, true=0.46570010922928873
prediction = 1.0, true=0.5399967709260678
prediction = 1.0, true=0.547442582465407
prediction = 1.0, true=0.469416250179268
prediction = 1.0, true=0.5325103704685856
prediction = 1.0, true=0.51347416785953
prediction = 1.0, true=0.5299388370